In [1]:
import pygame;
from pygame.locals import *
from pgu import engine
import random
import time
import game_board

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Tic_Tac_Toe(engine.Game):
    def __init__(self):
        super().__init__()
        pygame.mixer.pre_init(44100, -16, 2, 512)
        pygame.init()
        pygame.font.init()
        self.screen = pygame.display.set_mode((750,750), SWSURFACE)
        self.crono = pygame.time.Clock()
        self._init_state_machine()
        
    def _init_state_machine(self):
        self.menu_state = Menu(self)
        self.playervsCPU_state = PlayerVSCPU(self)
        self.result_state = Result(self)
        self.quit_state = engine.Quit(self)
    
    def change_state(self, transition=None):
        """
        Implements the state machine of the game.
        Given self.state and an optional parameter indicating 
        the kind of transition, computes and returns the new state
        """
        if self.state is self.menu_state:
            if transition == 'EXIT':
                new_state = self.quit_state
            elif transition == 'PlayerVSCPU':
                new_state = self.playervsCPU_state
            else:
                raise ValueError('Unknown transition indicator')
        elif self.state is self.playervsCPU_state:
            if transition == 'EXIT':
                new_state = self.quit_state
            elif transition == 'RESULT':
                new_state = self.result_state
            else:
                raise ValueError('Unknown transition indicator')
        elif self.state is self.result_state:
            if transition == 'EXIT':
                new_state = self.quit_state
            elif transition == 'RETRY':
                new_state = self.playervsCPU_state
                new_state.init()
            elif transition == 'MENU':
                new_state = self.menu_state
            else:
                raise ValueError('Unknown transition indicator')
        else:
            raise ValueError('Unknown game state value')
        return new_state
        
    def run(self):
        # Calls the main loop with the initial state
        # (self.menu, in this case)
        super().run(self.menu_state, self.screen)

##------------------------------------------------------Menu screen-------------------------------------------

class Menu(engine.State):
    def init(self):
        #self.image = pygame.image.load("env/menu_def.jpg") 
        #pygame.mixer.music.load("env/intro.ogg")
        #pygame.mixer.music.play()
        self.pvCPU_button = game_board.Button("buttons.png",(250,500))
        self.pvp_button = game_board.Button("buttons.png",(500,500))
        self.buttons = pygame.sprite.Group()
        self.buttons.add(self.pvCPU_button)
        self.buttons.add(self.pvp_button)
        
    def paint(self, s):
        s.fill((0, 0, 0))
        #rect = self.image.get_rect()
        #rect.center = s.get_rect().center
        #s.blit(self.image, rect)

    def event(self,e): 
        if e.type is KEYDOWN:
            if e.key == K_ESCAPE:
                return self.game.change_state('EXIT')
        elif e.type is pygame.MOUSEBUTTONDOWN:
            for b in list(self.buttons):
                    if b.rect.collidepoint(e.pos):
                        if  b == self.pvCPU_button:
                            return self.game.change_state('PlayerVSCPU')
    def loop(self):
        pass
    
    def update(self,screen):
        self.paint(screen)
        self.buttons.draw(screen)
        
        pygame.display.flip()
            

 ##------------------------------------------------------Player_vs_CPU screen ---------------------------------
class PlayerVSCPU(engine.State):
    def init(self):
        self.board = pygame.image.load("board.png")
        
        #Initiate the sprites
        self.cells = game_board.GameBoard()
                
#        self.sel_cell=pygame.sprite.GroupSingle()
        
        self.turn = random.choice([1,2])
        self.finish = False
        self._init = 0
    
    def paint(self, s):
        s.fill((0, 0, 0))
        board_rect = self.board.get_rect()
        board_rect.center = s.get_rect().center
        s.blit(self.board, board_rect)
    
    def switch_turn(self):
        self.actual_time=pygame.time.get_ticks()
        if self.turn==2:
            self.turn=1
        else:
            self.turn+=1
            self.actual_time=pygame.time.get_ticks()
    
    def CPU_turn(self):
#        if self.actual_time+2000<=pygame.time.get_ticks():
        counter = 0
        num_empty_cells = []
        for cell in self.cells.matrix:
            if cell == 'empty':
                num_empty_cells.append(counter)
                counter = counter+1
            else:
                counter = counter+1
        if num_empty_cells != []:
            chosen_cell = random.choice(num_empty_cells)
            self.CPU_cell = game_board.GameBoardCell(chosen_cell,'cross')
            self.cells.add(self.CPU_cell)
            self.cells.matrix[self.CPU_cell.pos] = self.CPU_cell.mark
            self.cells.positioning()
            
    def final(self):
        if (self.cells.matrix[0] == self.cells.matrix[1] and self.cells.matrix[0] == self.cells.matrix[2]) and self.cells.matrix[0] !='empty' and not self.finish:
            self.finish = True
            if self.cells.matrix[0] == 'circle':
                self.result = 'Player wins'
            else:
                self.result = 'CPU wins'
        elif (self.cells.matrix[3] == self.cells.matrix[4] and self.cells.matrix[3] == self.cells.matrix[5]) and self.cells.matrix[3] !='empty' and not self.finish:
            self.finish = True
            if self.cells.matrix[3] == 'circle':
                self.result = 'Player wins'
            else:
                self.winner = 'CPU wins'
        elif (self.cells.matrix[6] == self.cells.matrix[7] and self.cells.matrix[6] == self.cells.matrix[8]) and self.cells.matrix[6] !='empty' and not self.finish:
            self.finish = True
            if self.cells.matrix[6] == 'circle':
                self.result = 'Player wins'
            else:
                self.result = 'CPU wins'
        elif (self.cells.matrix[0] == self.cells.matrix[3] and self.cells.matrix[0] == self.cells.matrix[6]) and self.cells.matrix[0] !='empty' and not self.finish:
            self.finish = True
            if self.cells.matrix[0] == 'circle':
                self.result = 'Player wins'
            else:
                self.result = 'CPU wins'
        elif (self.cells.matrix[1] == self.cells.matrix[4] and self.cells.matrix[1] == self.cells.matrix[7]) and self.cells.matrix[1] !='empty' and not self.finish:
            self.finish = True
            if self.cells.matrix[1] == 'circle':
                self.result = 'Player wins'
            else:
                self.result = 'CPU wins'
        elif (self.cells.matrix[2] == self.cells.matrix[5] and self.cells.matrix[2] == self.cells.matrix[8]) and self.cells.matrix[2] !='empty' and not self.finish:
            self.finish = True
            if self.cells.matrix[2] == 'circle':
                self.result = 'Player wins'
            else:
                self.result = 'CPU wins'
        elif (self.cells.matrix[0] == self.cells.matrix[4] and self.cells.matrix[0] == self.cells.matrix[8]) and self.cells.matrix[0] !='empty' and not self.finish:
            self.finish = True
            if self.cells.matrix[0] == 'circle':
                self.result = 'Player wins'
            else:
                self.result = 'CPU wins'
        elif (self.cells.matrix[2] == self.cells.matrix[4] and self.cells.matrix[2] == self.cells.matrix[6]) and self.cells.matrix[2] !='empty' and not self.finish:
            self.finish = True
            if self.cells.matrix[2] == 'circle':
                self.result = 'Player wins'
            else:
                self.result = 'CPU wins'
        if 'empty' not in self.cells.matrix:
            self.result = 'Draw'
            self.finish = True
            
    def event(self,event):
        if event.type is KEYDOWN:
            if event.key == K_q:
                return self.game.change_state('EXIT')
        elif event.type is pygame.MOUSEBUTTONDOWN:
            if self.turn==1:
                if (event.pos[0]<=250 and event.pos[1]<=250) and self.cells.matrix[0] == 'empty':
                    self.cell1 = game_board.GameBoardCell(0,'circle')
                    self.cells.add(self.cell1)
                    self.cells.matrix[self.cell1.pos] = self.cell1.mark
                    self.cells.positioning()
                    self.switch_turn()
                elif ((250<event.pos[0]<=500) and event.pos[1]<=250) and self.cells.matrix[1] == 'empty':
                    self.cell2 = game_board.GameBoardCell(1,'circle')
                    self.cells.add(self.cell2)
                    self.cells.matrix[self.cell2.pos] = self.cell2.mark
                    self.cells.positioning()
                    self.switch_turn()
                elif (event.pos[0]>500 and event.pos[1]<=250) and self.cells.matrix[2] == 'empty':
                    self.cell3 = game_board.GameBoardCell(2,'circle')
                    self.cells.add(self.cell3)
                    self.cells.matrix[self.cell3.pos] = self.cell3.mark
                    self.cells.positioning()
                    self.switch_turn()
                elif (event.pos[0]<=250 and (250<event.pos[1]<=500)) and self.cells.matrix[3] == 'empty':
                    self.cell4 = game_board.GameBoardCell(3,'circle')
                    self.cells.add(self.cell4)
                    self.cells.matrix[self.cell4.pos] = self.cell4.mark
                    self.cells.positioning()
                    self.switch_turn()
                elif ((250<event.pos[0]<=500) and (250<event.pos[1]<=500)) and self.cells.matrix[4] == 'empty':
                    self.cell5 = game_board.GameBoardCell(4,'circle')
                    self.cells.add(self.cell5)
                    self.cells.matrix[self.cell5.pos] = self.cell5.mark
                    self.cells.positioning()
                    self.switch_turn()
                elif (event.pos[0]>500 and (250<event.pos[1]<=500)) and self.cells.matrix[5] == 'empty':
                    self.cell6 = game_board.GameBoardCell(5,'circle')
                    self.cells.add(self.cell6)
                    self.cells.matrix[self.cell6.pos] = self.cell6.mark
                    self.cells.positioning()
                    self.switch_turn()
                elif (event.pos[0]<=250 and event.pos[1]>500) and self.cells.matrix[6] == 'empty':
                    self.cell7 = game_board.GameBoardCell(6,'circle')
                    self.cells.add(self.cell7)
                    self.cells.matrix[self.cell7.pos] = self.cell7.mark
                    self.cells.positioning()
                    self.switch_turn()
                elif ((250<event.pos[0]<=500) and event.pos[1]>500) and self.cells.matrix[7] == 'empty':
                    self.cell8 = game_board.GameBoardCell(7,'circle')
                    self.cells.add(self.cell8)
                    self.cells.matrix[self.cell8.pos] = self.cell8.mark
                    self.cells.positioning()
                    self.switch_turn()
                elif (event.pos[0]>500 and event.pos[1]>500) and self.cells.matrix[8] == 'empty':
                    self.cell9 = game_board.GameBoardCell(8,'circle')
                    self.cells.add(self.cell9)
                    self.cells.matrix[self.cell9.pos] = self.cell9.mark
                    self.cells.positioning()
                    self.switch_turn()
                else:
                    print('That cell is already taken, click again')
    
    def loop(self):
        if self.turn == 2:
            self.CPU_turn()
            self.switch_turn()
        self.cells.update()
        self.final()
        if self.finish:
            if self.actual_time+2000<=pygame.time.get_ticks():
                return self.game.change_state('RESULT')
    
    def update(self,screen):
        self.paint(screen)
        self.cells.draw(screen)
        
        pygame.display.flip()

##------------------------------------------------------Result screen ---------------------------------

class Result(engine.State):
    def init(self):
        #self.image = pygame.image.load("env/menu_def.jpg") 
        #pygame.mixer.music.load("env/intro.ogg")
        #pygame.mixer.music.play()
        self.retry_button = game_board.Button("buttons.png",(250,500))
        self.menu_button = game_board.Button("buttons.png",(500,500))
        self.buttons = pygame.sprite.Group()
        self.buttons.add(self.retry_button)
        self.buttons.add(self.menu_button)
        
    def paint(self, s):
        s.fill((0, 0, 0))
        #rect = self.image.get_rect()
        #rect.center = s.get_rect().center
        #s.blit(self.image, rect)

    def event(self,e): 
        if e.type is KEYDOWN:
            if e.key == K_ESCAPE:
                return self.game.change_state('EXIT')
        elif e.type is pygame.MOUSEBUTTONDOWN:
            for b in list(self.buttons):
                    if b.rect.collidepoint(e.pos):
                        if  b == self.retry_button:
                            return self.game.change_state('RETRY')
                        elif b == self.menu_button:
                            return self.game.change_state('MENU')
    def loop(self):
        pass
    
    def update(self,screen):
        self.paint(screen)
        self.buttons.draw(screen)
        
        pygame.display.flip()
def main():
    game = Tic_Tac_Toe()
    game.run()


if __name__ == "__main__":
    main()